In [ ]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = 'IXP_ASN.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
ASNs=list(df['asn'])
TR_df=df[df['country']=="TR"]
ASNs=list(TR_df['asn'])

In [ ]:
len(ASNs)

In [ ]:
import itertools
import subprocess
import concurrent.futures

# Function to run grep command and return the result
def grep_adjacent_numbers(combination):
    number1, number2 = combination
    print(combination)
    cmd = f"bgpdump rib.20240130.2000.bz2 | grep -E 'ASPATH:.*\\b{number1} {number2}\\b|\\b{number2} {number1}\\b'"
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if result.stdout:
        return f"Combination {number1} {number2}:\n{result.stdout}\n"
    return None

# Function to generate all combinations and run grep for each using multithreading
def search_adjacent_as_numbers(numbers, output_file):
    combinations = list(itertools.combinations(numbers, 2))
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        # Submit all grep tasks to the executor
        future_to_combination = {executor.submit(grep_adjacent_numbers, combination): combination for combination in combinations}
        
        # Write results to the file as they complete
        with open(output_file, 'w') as out:
            for future in concurrent.futures.as_completed(future_to_combination):
                result = future.result()
                if result:
                    out.write(result)

numbers = [6939,6775]  # Example list of AS numbers
output_file = 'Outs/ixp_inpath.txt'

search_adjacent_as_numbers(numbers, output_file)